In [1]:
import pyvista as pv
from seagullmesh import Mesh3
import numpy as np
import pymeshfix
from ga_population import *
import numpy as np
from sklearn.decomposition import PCA
from torchvision.utils import make_grid
import torchvision.transforms.functional as F
from scipy.stats import pearsonr
import warnings

pms = [pms.cached() for pms in ProbeMeshSpec.defined()]

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
models = SourceModels.current_trained()

In [9]:
cp_pm_preds_file.stem

'cp_pm_preds'

In [3]:
cp_pm_preds_file = Path('cp_pm_preds.hdf')
if cp_pm_preds_file.exists():
    cp_pm_preds = pd.read_hdf(cp_pm_preds_file, key=cp_pm_preds_file.stem)
else:
    cp_pm_preds = models.get_predictions(pms)
    cp_pm_preds.to_hdf(cp_pm_preds_file)

  0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
cp_responses_file = Path('cp_responses.hdf')
if cp_responses_file.exists():
    cp_responses = pd.read_hdf(cp_responses_file, key=cp_responses_file.stem)    
else:
    cp_responses = models.cp_responses.loc[cp_pm_preds.index]
    cp_responses.to_hdf(cp_responses_file, key=cp_responses_file.stem)

In [ ]:
# TODO:
# Construct population vector dot products / angles for each stim. Which models are most predictive?


In [ ]:
rvals = np.zeros(cp_pm_preds.shape[1])
for i, (unit_idx, r_mesh) in enumerate(cp_pm_preds.items()):
    r_corpus = cp_responses.loc[:, unit_idx]
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        rvals[i] = pearsonr(r_mesh, r_corpus).statistic
rvals[np.isnan(rvals)] = -1
best_units = cp_responses.columns[np.argsort(rvals)[::-1]]

plt.hist(rvals)